<a href="https://colab.research.google.com/github/pnn99/detector_fraude_LLM/blob/main/detector_de_fraude.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud?resource=download


In [1]:
# !pip install pandas
# !pip install numpy
# !pip install scikit-learn

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score

In [3]:
df_credit = pd.read_csv('creditcard.csv')
df_credit.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
#verificação de tipos e valores nulos
df_credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [5]:
#verifica os tipos dos dados não fraude
df_nao_fraude = df_credit.Amount[df_credit.Class == 0]
df_nao_fraude.describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [6]:
#verifica os tipos dos dados de fraude
df_fraude = df_credit.Amount[df_credit.Class == 1]
df_fraude.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [7]:
# Contagem das classes (fraude e não fraude)
df_credit.Class.value_counts()

Class
0    284315
1       492
Name: count, dtype: int64

In [8]:
#Criação de um novo dataset para utilizar um undersampling para igualar a nossa base de dados
df_fraude = df_credit[df_credit.Class == 1]
df_nao_fraude = df_credit[df_credit.Class == 0]

df_undersampling = df_nao_fraude.sample(n=492)
df_undersampling

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
64066,51006.0,-0.563131,0.939091,1.636718,0.957925,-0.223117,0.352043,0.175727,0.048631,-0.228586,...,0.075267,0.318825,-0.248574,0.087781,-0.195232,0.582909,-0.286380,0.109937,4.54,0
69654,53518.0,1.137589,-0.610794,1.011308,0.726259,-1.447220,-0.558740,-0.617176,-0.093103,-0.518615,...,-0.175431,-0.008173,-0.077196,0.715495,0.466138,-0.251455,0.067140,0.050462,79.00,0
102895,68416.0,0.971364,0.022071,0.035595,0.749916,0.503603,1.565065,-0.489634,0.624818,0.253562,...,-0.004801,0.362728,0.014799,-1.068822,0.215818,0.521472,0.071557,0.018889,12.31,0
130123,79289.0,1.245902,-0.438715,0.884924,-0.712873,-1.321442,-0.855017,-0.657008,0.037432,2.009842,...,-0.026625,0.112438,-0.020673,0.375442,0.458016,-0.682432,0.085511,0.032626,1.00,0
254311,156672.0,-2.021985,-2.071490,-0.082834,0.215948,2.186855,-2.158256,0.839806,-0.675553,-0.139822,...,0.175716,1.581809,2.160039,0.122094,-0.475637,0.709361,0.263466,-0.246391,35.95,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53667,46093.0,0.818804,-1.140302,0.781427,-0.684917,-1.437188,-0.119068,-0.692549,0.322007,1.859857,...,0.174781,0.269856,-0.123259,0.201772,0.240498,-0.683984,0.062705,0.048849,169.00,0
172398,121085.0,-1.104583,0.248202,0.164581,-1.232676,1.127790,2.469643,0.070663,1.229500,-0.387727,...,0.272103,0.560119,0.093460,-1.392640,-0.375938,0.682613,-0.032994,0.020648,94.50,0
118219,74996.0,1.216414,0.233450,0.341719,0.923765,-0.214906,-0.392999,-0.049624,0.020480,-0.079070,...,-0.033084,-0.131790,-0.063327,-0.027004,0.534344,-0.433314,0.014046,0.010374,2.50,0
123060,76793.0,-0.735075,1.070093,1.271650,0.103375,0.034555,-1.195619,1.048044,-0.169070,-0.247329,...,0.138378,0.420061,-0.100886,0.693906,-0.128991,-0.575245,0.246633,0.231074,33.50,0


In [9]:
#concaternar dos dataframes
df_concatenado = pd.concat([df_undersampling, df_fraude], axis=0)
df_concatenado

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
64066,51006.0,-0.563131,0.939091,1.636718,0.957925,-0.223117,0.352043,0.175727,0.048631,-0.228586,...,0.075267,0.318825,-0.248574,0.087781,-0.195232,0.582909,-0.286380,0.109937,4.54,0
69654,53518.0,1.137589,-0.610794,1.011308,0.726259,-1.447220,-0.558740,-0.617176,-0.093103,-0.518615,...,-0.175431,-0.008173,-0.077196,0.715495,0.466138,-0.251455,0.067140,0.050462,79.00,0
102895,68416.0,0.971364,0.022071,0.035595,0.749916,0.503603,1.565065,-0.489634,0.624818,0.253562,...,-0.004801,0.362728,0.014799,-1.068822,0.215818,0.521472,0.071557,0.018889,12.31,0
130123,79289.0,1.245902,-0.438715,0.884924,-0.712873,-1.321442,-0.855017,-0.657008,0.037432,2.009842,...,-0.026625,0.112438,-0.020673,0.375442,0.458016,-0.682432,0.085511,0.032626,1.00,0
254311,156672.0,-2.021985,-2.071490,-0.082834,0.215948,2.186855,-2.158256,0.839806,-0.675553,-0.139822,...,0.175716,1.581809,2.160039,0.122094,-0.475637,0.709361,0.263466,-0.246391,35.95,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1
280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1
280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1
281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1


In [10]:
#ajustar o Index
df_concatenado.reset_index(drop=True, inplace=True)
df_concatenado

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,51006.0,-0.563131,0.939091,1.636718,0.957925,-0.223117,0.352043,0.175727,0.048631,-0.228586,...,0.075267,0.318825,-0.248574,0.087781,-0.195232,0.582909,-0.286380,0.109937,4.54,0
1,53518.0,1.137589,-0.610794,1.011308,0.726259,-1.447220,-0.558740,-0.617176,-0.093103,-0.518615,...,-0.175431,-0.008173,-0.077196,0.715495,0.466138,-0.251455,0.067140,0.050462,79.00,0
2,68416.0,0.971364,0.022071,0.035595,0.749916,0.503603,1.565065,-0.489634,0.624818,0.253562,...,-0.004801,0.362728,0.014799,-1.068822,0.215818,0.521472,0.071557,0.018889,12.31,0
3,79289.0,1.245902,-0.438715,0.884924,-0.712873,-1.321442,-0.855017,-0.657008,0.037432,2.009842,...,-0.026625,0.112438,-0.020673,0.375442,0.458016,-0.682432,0.085511,0.032626,1.00,0
4,156672.0,-2.021985,-2.071490,-0.082834,0.215948,2.186855,-2.158256,0.839806,-0.675553,-0.139822,...,0.175716,1.581809,2.160039,0.122094,-0.475637,0.709361,0.263466,-0.246391,35.95,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1
980,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1
981,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1
982,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1


In [11]:
#retirar dados para validação
df_val_nao_fraude = df_concatenado.head(5)
df_val_nao_fraude

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,51006.0,-0.563131,0.939091,1.636718,0.957925,-0.223117,0.352043,0.175727,0.048631,-0.228586,...,0.075267,0.318825,-0.248574,0.087781,-0.195232,0.582909,-0.286380,0.109937,4.54,0
1,53518.0,1.137589,-0.610794,1.011308,0.726259,-1.447220,-0.558740,-0.617176,-0.093103,-0.518615,...,-0.175431,-0.008173,-0.077196,0.715495,0.466138,-0.251455,0.067140,0.050462,79.00,0
2,68416.0,0.971364,0.022071,0.035595,0.749916,0.503603,1.565065,-0.489634,0.624818,0.253562,...,-0.004801,0.362728,0.014799,-1.068822,0.215818,0.521472,0.071557,0.018889,12.31,0
3,79289.0,1.245902,-0.438715,0.884924,-0.712873,-1.321442,-0.855017,-0.657008,0.037432,2.009842,...,-0.026625,0.112438,-0.020673,0.375442,0.458016,-0.682432,0.085511,0.032626,1.00,0
4,156672.0,-2.021985,-2.071490,-0.082834,0.215948,2.186855,-2.158256,0.839806,-0.675553,-0.139822,...,0.175716,1.581809,2.160039,0.122094,-0.475637,0.709361,0.263466,-0.246391,35.95,0


In [12]:
df_val_fraude = df_concatenado.tail(5)
df_val_fraude

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
979,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1
980,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1
981,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1
982,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1
983,170348.0,1.991976,0.158476,-2.583441,0.408670,1.151147,-0.096695,0.223050,-0.068384,0.577829,...,-0.164350,-0.295135,-0.072173,-0.450261,0.313267,-0.289617,0.002988,-0.015309,42.53,1


In [13]:
df_concatenado.drop(df_concatenado.head(5).index, inplace=True)
df_concatenado.drop(df_concatenado.tail(5).index, inplace=True)
df_concatenado

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
5,130524.0,1.530201,-2.139943,-0.255543,-0.496726,-1.579669,0.552842,-1.173124,0.211382,0.498566,...,0.229995,0.255277,0.063935,0.806111,-0.471320,-0.387518,-0.009720,0.008156,289.70,0
6,130432.0,-2.714791,-1.714888,1.369393,-1.031131,0.448966,-1.598001,-0.519842,0.562149,0.130573,...,0.433251,0.119719,-0.102404,0.106358,0.399961,1.289071,-0.211348,-0.333697,127.50,0
7,162493.0,-0.080261,0.413198,1.103048,-1.494951,0.834285,0.125379,0.668445,-0.154833,0.563265,...,-0.231153,-0.455427,-0.084340,0.030657,-0.211195,-0.411693,-0.127180,-0.183379,10.00,0
8,149152.0,-0.033109,0.554431,-0.456596,-0.263926,1.383758,-0.960382,0.902033,-0.371941,-0.114453,...,0.315026,1.107132,-0.210979,-0.466351,-0.403889,-0.190885,-0.178723,-0.044449,6.29,0
9,63566.0,-0.739143,1.117830,1.694701,-0.604023,0.496715,-0.471922,0.845872,0.032011,-0.965518,...,-0.200765,-0.695978,-0.253491,-0.033395,0.243459,0.021240,-0.005617,0.060489,1.98,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,165981.0,-5.766879,-8.402154,0.056543,6.950983,9.880564,-5.773192,-5.748879,0.721743,-1.076274,...,0.880395,-0.130436,2.241471,0.665346,-1.890041,-0.120803,0.073269,0.583799,0.00,1
975,166028.0,-0.956390,2.361594,-3.171195,1.970759,0.474761,-1.902598,-0.055178,0.277831,-1.745854,...,0.473211,0.719400,0.122458,-0.255650,-0.619259,-0.484280,0.683535,0.443299,39.90,1
976,166831.0,-2.027135,-1.131890,-1.135194,1.086963,-0.010547,0.423797,3.790880,-1.155595,-0.063434,...,-0.315105,0.575520,0.490842,0.756502,-0.142685,-0.602777,0.508712,-0.091646,634.30,1
977,166883.0,2.091900,-0.757459,-1.192258,-0.755458,-0.620324,-0.322077,-1.082511,0.117200,-0.140927,...,0.288253,0.831939,0.142007,0.592615,-0.196143,-0.136676,0.020182,-0.015470,19.95,1


In [14]:
#concatenar o dataframe de validação
df_val_total = pd.concat([df_val_nao_fraude, df_val_fraude], axis=0)
df_val_total.reset_index(inplace=True)

df_val_total_real = df_val_total.Class
df_val_total = df_val_total.drop(['index','Time','Class'], axis=1)
df_val_total

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,-0.563131,0.939091,1.636718,0.957925,-0.223117,0.352043,0.175727,0.048631,-0.228586,0.258187,...,0.127507,0.075267,0.318825,-0.248574,0.087781,-0.195232,0.582909,-0.286380,0.109937,4.54
1,1.137589,-0.610794,1.011308,0.726259,-1.447220,-0.558740,-0.617176,-0.093103,-0.518615,0.654761,...,-0.391847,-0.175431,-0.008173,-0.077196,0.715495,0.466138,-0.251455,0.067140,0.050462,79.00
2,0.971364,0.022071,0.035595,0.749916,0.503603,1.565065,-0.489634,0.624818,0.253562,-0.808505,...,-0.175025,-0.004801,0.362728,0.014799,-1.068822,0.215818,0.521472,0.071557,0.018889,12.31
3,1.245902,-0.438715,0.884924,-0.712873,-1.321442,-0.855017,-0.657008,0.037432,2.009842,-1.043276,...,-0.205519,-0.026625,0.112438,-0.020673,0.375442,0.458016,-0.682432,0.085511,0.032626,1.00
4,-2.021985,-2.071490,-0.082834,0.215948,2.186855,-2.158256,0.839806,-0.675553,-0.139822,0.063473,...,-0.354198,0.175716,1.581809,2.160039,0.122094,-0.475637,0.709361,0.263466,-0.246391,35.95
5,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,-5.587794,...,1.252967,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00
6,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,-3.232153,...,0.226138,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76
7,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,-3.463891,...,0.247968,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89
8,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,-5.245984,...,0.306271,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00
9,1.991976,0.158476,-2.583441,0.408670,1.151147,-0.096695,0.223050,-0.068384,0.577829,-0.888722,...,-0.017652,-0.164350,-0.295135,-0.072173,-0.450261,0.313267,-0.289617,0.002988,-0.015309,42.53


In [15]:
#verificar a distribuição dos dados fraude/não fraude
df_concatenado.Class.value_counts()

Class
0    487
1    487
Name: count, dtype: int64

In [16]:
#separar labels e features
X = df_concatenado.drop(['Time','Class'], axis=1)
Y = df_concatenado['Class']

In [17]:
print(X, Y)

           V1        V2        V3        V4        V5        V6        V7  \
5    1.530201 -2.139943 -0.255543 -0.496726 -1.579669  0.552842 -1.173124   
6   -2.714791 -1.714888  1.369393 -1.031131  0.448966 -1.598001 -0.519842   
7   -0.080261  0.413198  1.103048 -1.494951  0.834285  0.125379  0.668445   
8   -0.033109  0.554431 -0.456596 -0.263926  1.383758 -0.960382  0.902033   
9   -0.739143  1.117830  1.694701 -0.604023  0.496715 -0.471922  0.845872   
..        ...       ...       ...       ...       ...       ...       ...   
974 -5.766879 -8.402154  0.056543  6.950983  9.880564 -5.773192 -5.748879   
975 -0.956390  2.361594 -3.171195  1.970759  0.474761 -1.902598 -0.055178   
976 -2.027135 -1.131890 -1.135194  1.086963 -0.010547  0.423797  3.790880   
977  2.091900 -0.757459 -1.192258 -0.755458 -0.620324 -0.322077 -1.082511   
978 -1.374424  2.793185 -4.346572  2.400731 -1.688433  0.111136 -0.922038   

           V8        V9       V10  ...       V20       V21       V22  \
5  

In [18]:
#separar entre dados de treino e teste
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state = 42)

In [19]:
# --- Importação dos novos modelos e do StackingClassifier ---
from sklearn.ensemble import StackingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier # Adicionei o KNN como opção também
from sklearn.metrics import classification_report # Para uma análise mais detalhada

# ---------------------------------------------------------------------------------
# PASSO 1: Definir os modelos de base (Nível 0 - Os "Especialistas")
# ---------------------------------------------------------------------------------
# Criamos uma lista de tuplas com o nome e o objeto de cada modelo.
base_models = [
    ('logistic_regression', LogisticRegression(max_iter=1000, random_state=42)),
    ('random_forest', RandomForestClassifier(random_state=42)),
    ('svc', SVC(probability=True, random_state=42)), # probability=True é necessário para o Stacking
    ('gradient_boosting', GradientBoostingClassifier(random_state=42))
]

# ---------------------------------------------------------------------------------
# PASSO 2: Definir o Metaclassificador (Nível 1 - O "Gerente")
# ---------------------------------------------------------------------------------
# Usaremos uma Regressão Logística para combinar as previsões dos especialistas.
# É uma escolha robusta que evita overfitting nesta etapa.
meta_classifier = LogisticRegression(random_state=42)

# ---------------------------------------------------------------------------------
# PASSO 3: Criar o Stacking Classifier
# ---------------------------------------------------------------------------------
# Juntamos tudo no StackingClassifier.
# O cv=5 garante que a validação cruzada será usada para treinar os modelos de base,
# o que é crucial para evitar vazamento de dados (data leakage).
stacking_model = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_classifier,
    cv=5
)

# ---------------------------------------------------------------------------------
# PASSO 4: Treinar e Avaliar o modelo Stacking
# ---------------------------------------------------------------------------------
print("--- Treinando o StackingClassifier ---")
stacking_model.fit(x_train, y_train)
y_pred_stacking = stacking_model.predict(x_test)
acc_stacking = accuracy_score(y_test, y_pred_stacking)
print(f"Acurácia do StackingClassifier: {acc_stacking * 100:.2f}%\n")
print("Relatório de Classificação do StackingClassifier:")
print(classification_report(y_test, y_pred_stacking))


# ---------------------------------------------------------------------------------
# PASSO 5: Treinar e Avaliar cada modelo de base para comparação
# ---------------------------------------------------------------------------------
print("\n--- Acurácia dos Modelos de Base (Individuais) ---")
for name, model in base_models:
    model.fit(x_train, y_train)
    y_pred_base = model.predict(x_test)
    acc_base = accuracy_score(y_test, y_pred_base)
    print(f"Acurácia do modelo '{name}': {acc_base * 100:.2f}%")

--- Treinando o StackingClassifier ---
Acurácia do StackingClassifier: 95.38%

Relatório de Classificação do StackingClassifier:
              precision    recall  f1-score   support

           0       0.92      0.99      0.96        98
           1       0.99      0.92      0.95        97

    accuracy                           0.95       195
   macro avg       0.96      0.95      0.95       195
weighted avg       0.96      0.95      0.95       195


--- Acurácia dos Modelos de Base (Individuais) ---
Acurácia do modelo 'logistic_regression': 95.90%
Acurácia do modelo 'random_forest': 95.38%
Acurácia do modelo 'svc': 81.03%
Acurácia do modelo 'gradient_boosting': 94.36%


In [20]:
# --- Validação Comparativa (1ª Rodada - Modelos Padrão) ---

# 1. Criar o DataFrame para os resultados desta rodada
df_validacao_rodada1 = pd.DataFrame({'Real': df_val_total_real})

# 2. Reinstanciar e treinar os modelos com parâmetros padrão
#    para garantir que estamos usando as versões não otimizadas.

# Lista dos modelos de base padrão
base_models_padrao = [
    ('logistic_regression', LogisticRegression(max_iter=1000, random_state=42)),
    ('random_forest', RandomForestClassifier(random_state=42)),
    ('svc', SVC(probability=True, random_state=42)),
    ('gradient_boosting', GradientBoostingClassifier(random_state=42))
]

# Treinando cada modelo individualmente e fazendo a previsão
print("--- Gerando previsões para os modelos padrão individuais ---")
for name, model in base_models_padrao:
    model.fit(x_train, y_train)
    df_validacao_rodada1[f'Prev ({name})'] = model.predict(df_val_total)

# 3. Treinar o StackingClassifier com os modelos padrão e fazer a previsão
print("--- Gerando previsões para o StackingClassifier padrão ---")
stacking_model_padrao = StackingClassifier(
    estimators=base_models_padrao,
    final_estimator=LogisticRegression(random_state=42),
    cv=5,
    n_jobs=-1
)
stacking_model_padrao.fit(x_train, y_train)
df_validacao_rodada1['Prev (STACKING PADRÃO)'] = stacking_model_padrao.predict(df_val_total)

# 4. Reorganizar as colunas para uma melhor visualização e exibir o DataFrame
colunas_ordenadas = [
    'Real',
    'Prev (logistic_regression)',
    'Prev (random_forest)',
    'Prev (svc)',
    'Prev (gradient_boosting)',
    'Prev (STACKING PADRÃO)'
]
df_validacao_rodada1 = df_validacao_rodada1[colunas_ordenadas]

print("\n--- Painel de Validação (1ª Rodada - Modelos Padrão) ---")
df_validacao_rodada1

--- Gerando previsões para os modelos padrão individuais ---
--- Gerando previsões para o StackingClassifier padrão ---

--- Painel de Validação (1ª Rodada - Modelos Padrão) ---


,Real,Prev (logistic_regression),Prev (random_forest),Prev (svc),Prev (gradient_boosting),Prev (STACKING PADRÃO)
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
5,1,1,1,1,1,1
6,1,1,1,0,1,1
7,1,1,1,0,1,1
8,1,1,1,0,1,1
9,1,0,0,0,0,0


In [21]:
from sklearn.model_selection import GridSearchCV

# --- PASSO 1: Otimização do RandomForestClassifier ---

print("--- Iniciando a Otimização para o RandomForestClassifier ---")

# Definindo o "cardápio" de parâmetros que o GridSearchCV vai testar
param_grid_rf = {
    'n_estimators': [50, 100, 200],  # Número de árvores
    'max_depth': [10, 20, 30],         # Profundidade máxima
    'criterion': ['gini', 'entropy']     # Critério de divisão
}

# Configurando o GridSearchCV
# cv=5 significa que usaremos validação cruzada de 5 folds
# n_jobs=-1 usa todos os processadores disponíveis para acelerar o processo
grid_search_rf = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid_rf,
    cv=5,
    n_jobs=-1,
    verbose=2 # O verbose=2 mostra o progresso do treinamento
)

# Rodando a busca pelos melhores parâmetros
grid_search_rf.fit(x_train, y_train)

print("\n--- Otimização do RandomForestClassifier Concluída! ---")
print(f"Melhores parâmetros encontrados: {grid_search_rf.best_params_}")

# Guardando o melhor modelo encontrado
best_rf = grid_search_rf.best_estimator_

# --- PASSO 2: Otimização do SVC ---

print("\n--- Iniciando a Otimização para o SVC ---")

# "Cardápio" de parâmetros para o SVC
param_grid_svc = {
    'C': [0.1, 1, 10],            # Parâmetro de regularização
    'gamma': ['scale', 'auto'],     # Coeficiente do kernel
    'kernel': ['rbf']               # Vamos focar no kernel RBF que é muito poderoso
}

# Configurando o GridSearchCV para o SVC
grid_search_svc = GridSearchCV(
    estimator=SVC(probability=True, random_state=42),
    param_grid=param_grid_svc,
    cv=5,
    n_jobs=-1,
    verbose=2
)

# Rodando a busca
grid_search_svc.fit(x_train, y_train)

print("\n--- Otimização do SVC Concluída! ---")
print(f"Melhores parâmetros encontrados: {grid_search_svc.best_params_}")

# Guardando o melhor modelo SVC
best_svc = grid_search_svc.best_estimator_

--- Iniciando a Otimização para o RandomForestClassifier ---
Fitting 5 folds for each of 18 candidates, totalling 90 fits

--- Otimização do RandomForestClassifier Concluída! ---
Melhores parâmetros encontrados: {'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 50}

--- Iniciando a Otimização para o SVC ---
Fitting 5 folds for each of 6 candidates, totalling 30 fits

--- Otimização do SVC Concluída! ---
Melhores parâmetros encontrados: {'C': 1, 'gamma': 'auto', 'kernel': 'rbf'}


In [ ]:
# --- PASSO FINAL: Montando e Avaliando o StackingClassifier OTIMIZADO ---

print("--- Montando o StackingClassifier com os melhores parâmetros ---")

# 1. Definir a lista de modelos de base OTIMIZADA
base_models_optimized = [
    ('logistic_regression', LogisticRegression(max_iter=1000, random_state=42)),
    # Usando os MELHORES parâmetros encontrados pelo GridSearchCV
    ('random_forest', RandomForestClassifier(
        criterion='entropy',
        max_depth=10,
        n_estimators=100,
        random_state=42
    )),
    # Usando os MELHORES parâmetros para o SVC
    ('svc', SVC(
        C=10,
        gamma='scale',
        kernel='rbf',
        probability=True, # Lembre-se que probability=True é essencial
        random_state=42
    )),
    ('gradient_boosting', GradientBoostingClassifier(random_state=42))
]

# 2. Definir o Metaclassificador (pode ser o mesmo de antes)
meta_classifier_final = LogisticRegression(random_state=42)

# 3. Criar o Stacking Classifier OTIMIZADO
stacking_model_optimized = StackingClassifier(
    estimators=base_models_optimized,
    final_estimator=meta_classifier_final,
    cv=5,
    n_jobs=-1
)

# 4. Treinar e Avaliar o modelo final
print("--- Treinando o StackingClassifier OTIMIZADO ---")
stacking_model_optimized.fit(x_train, y_train)
y_pred_stacking_optimized = stacking_model_optimized.predict(x_test)
acc_stacking_optimized = accuracy_score(y_test, y_pred_stacking_optimized)

print("\n--- RESULTADO FINAL ---")
print(f"Acurácia do StackingClassifier OTIMIZADO: {acc_stacking_optimized * 100:.2f}%\n")
print("Relatório de Classificação do StackingClassifier OTIMIZADO:")
print(classification_report(y_test, y_pred_stacking_optimized))

# Vamos também reavaliar a performance do SVC otimizado sozinho para ver o quanto ele melhorou
print("\n--- Verificando a melhoria do SVC Otimizado (Individual) ---")
svc_optimized_solo = SVC(C=10, gamma='scale', kernel='rbf', random_state=42)
svc_optimized_solo.fit(x_train, y_train)
acc_svc_optimized = accuracy_score(y_test, svc_optimized_solo.predict(x_test))
print(f"Acurácia do SVC OTIMIZADO: {acc_svc_optimized * 100:.2f}%")

--- Montando o StackingClassifier com os melhores parâmetros ---
--- Treinando o StackingClassifier OTIMIZADO ---

--- RESULTADO FINAL ---
Acurácia do StackingClassifier OTIMIZADO: 95.38%

Relatório de Classificação do StackingClassifier OTIMIZADO:
              precision    recall  f1-score   support

           0       0.92      0.99      0.96        98
           1       0.99      0.92      0.95        97

    accuracy                           0.95       195
   macro avg       0.96      0.95      0.95       195
weighted avg       0.96      0.95      0.95       195


--- Verificando a melhoria do SVC Otimizado (Individual) ---
Acurácia anterior do SVC (padrão): 82.56%
Acurácia do SVC OTIMIZADO: 88.72%


In [23]:
# --- Validação Final Comparativa ---

# 1. Vamos criar um DataFrame para armazenar os resultados.
# A primeira coluna é o valor real, que já separamos no início.
df_validacao = pd.DataFrame({'Real': df_val_total_real})


# 2. Agora, vamos adicionar a previsão de cada um dos nossos modelos como uma nova coluna.
#    Usaremos os modelos já treinados nas etapas anteriores.

# Modelo de Regressão Logística (o primeiro que treinamos)
lr_final = LogisticRegression(max_iter=1000, random_state=42)
lr_final.fit(x_train, y_train)
df_validacao['Prev (Logística)'] = lr_final.predict(df_val_total)

# Melhor RandomForest (encontrado pelo GridSearchCV)
# A variável 'best_rf' já foi treinada pelo GridSearchCV
df_validacao['Prev (RandomForest Otimizado)'] = best_rf.predict(df_val_total)

# Melhor SVC (encontrado pelo GridSearchCV)
# A variável 'best_svc' já foi treinada pelo GridSearchCV
df_validacao['Prev (SVC Otimizado)'] = best_svc.predict(df_val_total)

# Modelo Stacking Otimizado (nosso modelo final)
df_validacao['Prev (STACKING OTIMIZADO)'] = stacking_model_optimized.predict(df_val_total)


# 3. Exibir o DataFrame comparativo
print("--- Painel de Validação Final ---")
print("Comparando o valor Real com a previsão de cada modelo:")
df_validacao

--- Painel de Validação Final ---
Comparando o valor Real com a previsão de cada modelo:


,Real,Prev (Logística),Prev (RandomForest Otimizado),Prev (SVC Otimizado),Prev (STACKING OTIMIZADO)
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0
5,1,1,1,1,1
6,1,1,1,1,1
7,1,1,1,1,1
8,1,1,1,1,1
9,1,0,0,0,0
